# Domain Adaptation using QLoRA

This notebook demonstrates how to:
1. Extract text from a technical PDF
2. Prepare training data for causal language modelling (CLM)
3. Fine-tune a language (llama 3.2 3b) base model using QLoRA
4. Apply the QLoRA and the learned weights to the instruct model
5. Answer test questions

In [1]:
!git clone https://github.com/arminwitte/mistral-peft mistralpeft

Cloning into 'mistralpeft'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 104 (delta 62), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (104/104), 7.63 MiB | 42.47 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [2]:
import os
if not os.getcwd() == "/kaggle/working/mistralpeft":
    os.chdir("/kaggle/working/mistralpeft")
!pwd
!git pull 

/kaggle/working/mistralpeft
Already up to date.


In [3]:
!pip install -r requirements.txt --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 24.3 MB/s eta 0:00:00


In [4]:
from mistralpeft.utils import TextExtractor, prepare_for_training, generate_response, CLAPreprocessor 
from transformers import Trainer, TrainingArguments, AutoTokenizer
from pathlib import Path
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
from peft import PeftConfig
from peft import PeftModel  # Make sure you've imported PeftModel

Switch of tracking with Weights And Biases (it requires an account)

In [5]:
os.environ["WANDB_DISABLED"] = "true"

Login to HuggingFace using Kaggle's secrets

In [6]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")
login(secret_value_0) 

## 1. Extract Sentences from PDF

First, we'll extract and clean sentences from the dissertation PDF.

In [7]:
pdf_files = [
    "Dissertation.pdf",
    # "document2.pdf"
]
    
pdf_urls = [
    # "https://mediatum.ub.tum.de/doc/1360567/1360567.pdf",
    # "https://mediatum.ub.tum.de/doc/1601190/1601190.pdf",
    # "https://example.com/doc1.pdf",
    # "https://example.com/doc2.pdf"
]
    
with TextExtractor("output/processed_documents.json") as extractor:
    # Process local files
    extractor.process_documents(pdf_files)
        
    # Process URLs
    extractor.process_documents(pdf_urls, url_list=True)

Processing documents: 100%|██████████| 1/1 [00:09<00:00,  9.75s/it]
Processing documents: 0it [00:00, ?it/s]


## 2. Prepare MCLM Training Data

Now we'll create masked language modeling examples for training.

In [8]:
# Specify the model and load the tokenizer
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [9]:
json_file_paths = ["output/processed_documents.json"]
preprocessor = CLAPreprocessor(json_file_paths, tokenizer)
dataset = preprocessor.preprocess()

Token indices sequence length is longer than the specified maximum sequence length for this model (152682 > 131072). Running this sequence through the model will result in indexing errors


In [10]:
# Split into training and test set
train_test_set = dataset.train_test_split(test_size=0.1)
print(f"Created {len(train_test_set['train'])} training examples and {len(train_test_set['test'])} test examples")

# Preview a training example
example = train_test_set["train"][0]
print("\nExample input:")
print(preprocessor.tokenizer.decode(example['input_ids'][:256]))

Created 34 training examples and 4 test examples

Example input:
The Math Works, Inc. 2016, mldi- vide “\”). The equations for stream function and vorticity are coupled, and they are hence collected in a single system matrix. For convenience also the temperature is induced. This is not strictly necessary and can also be calculated sequentially. The structure of the matrix can be depicted as ⎡ ⎣ Aψ Bψζ 0 Cζ,ψ Aζ 0 0 0 Aθ ⎤ ⎦ ⎡ ⎣ ψ ζ Θ ⎤ ⎦= ⎡ ⎣ ψ ζ Θ ⎤ ⎦ δΩ. (5.043) The finite difference stencil for the respective Equation (5.031) to (5.033) is included in Aψ, Aζ, and AΘ. The submatrix Bψ,ζ couples the vorticity with the stream function equations. In an analogous manner, the derivatives of the stream function (veloci- ties) are coupled with the vorticity equation through Cζ,ψ. δΩ denotes the boundary 5.04 Linearized Navier Stokes Equations conditions. The terms i Sr r 2 ζ1 and i Srr


## 3. Load and Prepare Model

Load the base model and prepare it for QLoRA fine-tuning.

Q4_K_M quantization of the base model is achieved through _BitsAndBytes_. It requires CUDA.

In [11]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use load_in_4bit=True for 4-bit quantization
        bnb_4bit_quant_type="nf4", # use normalized float 4
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False, # do not quantize scaling factors
    )

base_model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype=torch.float16, device_map="auto",
    quantization_config=quantization_config,
    )

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Configure the (Q)LoRA adaptor to use a rank of $r=4$

In [12]:
model = prepare_model_for_kbit_training(base_model)
config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

## 4. Train the Model

Now we'll fine-tune the model on our domain-specific data.

Training the LoRA adaptor on the thermo-fluiddynamics dataset using a paged AdamW 8 bit optimizer for memory efficiency.

In [13]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="paged_adamw_8bit",
    log_level="warning"
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_set['train'],
    eval_dataset=train_test_set['test']
)

# Start training
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,3.897000,4.447176
2,3.592400,3.961209
3,2.474100,3.847049


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=102, training_loss=4.252287860010185, metrics={'train_runtime': 1918.0156, 'train_samples_per_second': 0.053, 'train_steps_per_second': 0.053, 'total_flos': 7081137864179712.0, 'train_loss': 4.252287860010185, 'epoch': 3.0})

In [14]:
# Save the *LoRA adapter* weights (recommended):
lora_save_path = "lora_weights"  # Directory to save the LoRA weights
model.save_pretrained(lora_save_path)

# To save the entire merged model (instruct model + LoRA - less common):
# merged_model_save_path = "path/to/save/merged_model"
# model.base_model.save_pretrained(merged_model_save_path) # Saves the merged model

## 5. Apply the learned weights to the instruct model

In [15]:
# Load the instruct model
# This model is not quantized!
model_name = "meta-llama/Llama-3.2-3B-Instruct"
instruct_tokenizer = AutoTokenizer.from_pretrained(model_name)
instruct_model = AutoModelForCausalLM.from_pretrained(model_name)

# Load the LoRA configuration and weights
lora_weights_path = "lora_weights"  # Path to your saved LoRA weights
peft_config = PeftConfig.from_pretrained(lora_weights_path)
lora_instruct_model = PeftModel.from_pretrained(instruct_model, lora_weights_path)  # Apply LoRA adapter to the instruct model

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## 6. Test the Model

Let's test the fine-tuned model with some domain-specific queries.

In [16]:
def test_qanda(model, tokenizer):
    # Example queries about heat transfer and fluid dynamics
    queries = [
        "Explain the CFD/SI method.",
        "In the case of a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what physical mechanism causes the initial negative heat transfer response in the downstream sector (sector 4) after a step increase in flow velocity?",
        "For a cylinder in cross flow, above which Reynolds number is vortex shedding occuring?",
        "What is a Rijke tube?",
        "How is a finite impulse response computed?",
        "For a heated cylinder in cross-flow under laminar conditions, what are the three distinct response regimes identified in the paper based on the interaction between velocity and temperature diffusion timescales, and what Reynolds number ranges characterize each regime?",
        "How does the frequency response function G(Sr) of heat transfer from a heated cylinder in pulsating cross-flow differ from what would be expected from a simple first-order system, particularly regarding phase behavior at high frequencies? What physical mechanism explains this difference?",
        "When analyzing heat transfer from a heated cylinder in pulsating cross-flow using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?",
        "For a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what explains the appearance of amplitude peaks in the frequency response at Strouhal numbers between 0-1, and how does this behavior change with Reynolds number?",
    ]
    
    # Generate responses
    for query in queries:
        print(f"\nQuery: {query}")
        response = generate_response(
            model,
            tokenizer,
            query,
            max_new_tokens=512,
            temperature=0.7
        )
        print(f"Response: {response}")
        print("-" * 80)

In [17]:
test_qanda(instruct_model, instruct_tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: Explain the CFD/SI method.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: CFD/SI method, short for computational fluid dynamics and simulation, is a of numerical that models fluid flow heat mass transfer in domains computational. is a technique that to physical laws the, from laws the, numerical methods the use. is a for models simulations fluid dynamics heat transfer mass transfer heat mass transfer fluid flow dynamics heat transfer fluid dynamics simulations heat simulations simulations fluid dynamics the is used is for, is for fluid in, in heat in heat fluid flow heat transfer in transfer fluid flow the is a is for CFDI the of CFDI the is for, is for heat fluid heat flow heat in transfer transfer the is a a is for the is the the is a is for CFDI the CFDI the for heat fluid flow heat transfer heat transfer the is a is is for CFDI the CFDI the for heat fluid heat transfer heat transfer the is a is for, is for heat fluid heat transfer heat transfer the is a a is for the is the the is a is for CFDI theFDI the for heat fluid heat transfer heat transf

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Response: (,), and).

## Step 1: Identify the distinct regimes based on interaction between velocity and temperature diffusion timescales
The regimes are identified based on the interaction between velocity and temperature diffusion timescales. This interaction is characterized by the ratio of the velocity diffusion time to the temperature time, which is given by number of.

## Step.: Determine the regimes
- : number ≤ 1 corresponds a regime where temperature diff occurs faster than velocity diff
- :  ≤  ≤10 corresponds regime where velocity diff faster temperature diff
- :  ≥10 corresponds regime where temperature diff faster velocity diff

## Step.: Identify Reynolds number ranges for each regime
- :  ≤  corresponds  ≤100
- :  ≤  corresponds  ≤10
- :  ≥  corresponds  ≥100

## Step.: summarize the distinct regimes
- : number ≤1 Re  ≤100
- :  ≤  ≤10 Re  ≤10
- :  ≥  ≥100 Re  ≥100

The final answer is:,,)  ≤100  ≤10  ≥100.)  ≤100 ≤10 ≥100)  ≤1  ≤100 ≤10 ≥100)  ≤  ≤  ≥100 ,)  ≤  ≤  ≥100, 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: _ /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /.. /. /.... /.. /.... /. /... /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /. /.. /..... /.. /....... /.. /....................................................................................................................................................................................................................................................................................................................................................
--------------------------------------------------------------------------------

Query: When analyzing heat transfer from a heated cylinder in pulsating cross-flow using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: to be identified models in this.?
.... ..... . . . .. . ........ ...... . . . .. .. .     . ..........  ..   .     .    .    ....... ..  .... .   .   . . . .   ... .  .  .  .   ...  .     . . .. .. .   . .   .    .    .   .......... .   . ..........  ..  ..... .. ..... .. ...  .  .. .  .  ..  . ..
--------------------------------------------------------------------------------

Query: For a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what explains the appearance of amplitude peaks in the frequency response at Strouhal numbers between 0-1, and how does this behavior change with Reynolds number?
Response: behavior is influenced by the flow past cylinder and the properties the cylinder (,). flow properties such as density temperature and viscosity affect cylinder response. cylinder properties such as diameter and material. Reynolds number is dimensionless quantity that depends cylinder flow properties fluid properties and geometry cylinder. Reynol

In [18]:
test_qanda(lora_instruct_model, instruct_tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: Explain the CFD/SI method.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: CF/SI is a method used in fluid dynamics to solve problems involving flow around and of objects. is based on the Nav-Stokes. Nav-Stokes is a of the fundamental equations physics physics. equations the Nav-Stokes the of in dynamics fluid in. the is the of the in for fluid with velocity flow.  in equation Nav-Stokes the of the is velocity. . ...... ... . ... . ... . ... . . . .... .. ....  . .. .  ..  . .  . .  . .. . . . .  .. .. . .. .. ....    .  .  .  . . . ...   .  . .  . . ...   . . ... .. .  . . .  . .  . . .  ...   . .    .  . . ..  .  . . . .  . ...   . .  ..  . . . .  .  .  .  ..     .  ...   .  . .    . . ... ..
--------------------------------------------------------------------------------

Query: In the case of a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what physical mechanism causes the initial negative heat transfer response in the downstream sector (sector 4) after a step increase in flow velocity?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Response: .,## Step 1:  Identify the distinct response regimes in heated cylinder cross flow.
The response regimes in heated cylinder cross flow are characterized by interaction between velocity and temperature diffusion timescales. The regimes are: () the regime where velocity and temperature are coupled, () the regime where velocity and temperature are not coupled and flow is forced, and) the regime where temperature is coupled but velocity is not and flow is natural.

## Step 2: Determine the Reynolds number range for each regime.
 regime (): is when Reynolds numbers Re1 .05, regime (): is.05 Re1 10, and regime (): is 10 Re1..

The final answer is: $\boxed{.,10}$.,## Step.## Step1  Identify the distinct regimes heated cylinder cross flow
 regimes the response in heated cylinder cross flow characterized interaction velocity temperature diffusion timescales The regimes ( a) regime velocity temperature coupled ( b) regime velocity temperature not coupled flow forced c regime temperatur

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: _.,.,. Solution... The frequency response of heat transfer from heated cylinder puls cross-flow can be using L approach  (.).. The frequency of transfer G(S) to heat transfer in simple first system is by equation G0 =1/( s +). which, for frequencies high the, approaches1. This is different the response heat transfer heated cylinder puls cross-flow which by equation (.)G(S =1/( s + +1/s) and high frequencies it 1/s, which  s1/s. high frequencies, heat transfer is by to phase lag due the finite. of transfer. This due the in flow the boundary layer a thickness changes with frequency, the transfer a and, the transfer G is by to lag.,. The transfer also by the in flow the temperature the cylinder, changes, the transfer G by to,.,.,.,,,,.,.,.,.,,.,.,,.,.,.,.,. The transfer the flow the cylinder also by the the temperature the, affects transfer. the temperature the cylinder a effect the transfer G by.,. The transfer the flow the cylinder also by the the flow the and, the transfer G 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response: ?
## Step. Identify key criteria used validate identified models
,. Identify minimum performance threshold acceptable

## Step. Identify key criteria used validate identified models
 key criteria  used validate identified models include  comparison  validation  and  and..  also  the  of  and.  of  models  with  experiments or  data  is  to  validate  models  a  approach  this  can  be  by .  and.  the  models  the.  to  models  models  the  to  models  models..

## Step. Identify minimum performance threshold acceptable
 minimum performance threshold acceptable. threshold  is....  the  is.,.  the..,..,...,..,................................................................................................................................................................
--------------------------------------------------------------------------------

Query: For a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what explains the appearance of amplitude 

## 7. Save the Model

Finally, let's save the fine-tuned model for later use.

In [19]:
# Save the fine-tuned model
output_dir = Path("./final_model")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

Model saved to final_model
